In [33]:
import pandas as pd

df1 = pd.read_csv("data_scientist@austin#2023-07-20.csv")
df2 = pd.read_csv("data_scientist@chicago#2023-07-20.csv")
df3 = pd.read_csv("data_scientist@dallas#2023-07-20.csv")
df4 = pd.read_csv("data_scientist@houston#2023-07-20.csv")
df5 = pd.read_csv("data_scientist@los_angeles#2023-07-20.csv")
df6 = pd.read_csv("data_scientist@new_york#2023-07-20.csv")
df7 = pd.read_csv("data_scientist@philadelphia#2023-07-20.csv")
df8 = pd.read_csv("data_scientist@phoenix#2023-07-20.csv")
df9 = pd.read_csv("data_scientist@san_antonio#2023-07-20.csv")
df10 = pd.read_csv("data_scientist@san_diego#2023-07-20.csv")

df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index=True)

Chose data scientists because the variance is larger. 

In [25]:
import numpy as np
import nltk
nltk.download('stopwords')
import re
import os

from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

stopwords = nltk.corpus.stopwords.words('english')


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        1094 non-null   object
 1   company          1091 non-null   object
 2   job_location     1094 non-null   object
 3   post_date        1026 non-null   object
 4   salary           813 non-null    object
 5   job_url          1094 non-null   object
 6   job_description  1056 non-null   object
dtypes: object(7)
memory usage: 60.0+ KB


In [46]:
salary_df = df[df['salary'].notnull()]

salary_list = df['salary'].unique().tolist()

print(salary_list)

salary_df.info()

[nan, '$7,349 - $10,000 a month', '$109,000 - $166,000 a year', '$151,736 - $178,900 a year', 'Full-time', '$96,900 - $234,300 a year', '$99,900 - $168,400 a year', '$136,000 - $218,900 a year', '$249,600 - $299,500 a year', '$45 - $50 an hour', '$80,000 - $93,000 a year', '$135,100 - $196,100 a year', '$40 - $45 an hour', '$120,000 - $205,000 a year', 'Full-time +1', '$120,000 - $160,000 a year', '$40,000 a year', '$136,750 - $232,630 a year', '$80,985 - $122,489 a year', '$24.04 an hour', '$84,500 - $204,600 a year', '$152,000 - $339,250 a year', '$93,000 - $175,000 a year', '$77,800 - $116,700 a year', '$260,500 - $313,000 a year', '$65,000 - $70,000 a year', '$104,000 - $160,000 a year', '$75,900 - $141,456 a year', '$93,200 - $139,800 a year', '$131,000 - $203,000 a year', '$61,000 - $90,000 a year', '$127,000 - $140,025 a year', '$101,000 - $131,000 a year', 'Contract', '$92,800 - $123,800 a year', '$66,000 - $87,500 a year', '$83,400 - $104,200 a year', '$70,000 - $95,000 a year

In [49]:
def salary_base(x):
    y=""
    if "year" in str(x):
        y = "Yearly"
    if "hour" in str(x):
        y = "Hourly"
    if "day" in str(x):
        y = "Daily"
    elif "month" in str(x):
        y = "Monthly"
    
    return y

salary_df['Rate_by'] = salary_df['salary'].apply(salary_base)

/var/folders/hx/4w9b78g15zj3qxgwfj_1p8y00000gn/T/ipykernel_84645/2365099728.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_df['Rate_by'] = salary_df['salary'].apply(salary_base)


In [62]:
salary_df = salary_df[salary_df['Rate_by']!=""]
salary_df.info()

indexNames = salary_df[salary_df['Rate_by']==""].index
 
salary_df.drop(indexNames , inplace=True)
salary_df.reset_index(drop=True)

salary_df.groupby('Rate_by').salary.count()

<class 'pandas.core.frame.DataFrame'>
Index: 460 entries, 2 to 1093
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        460 non-null    object
 1   company          458 non-null    object
 2   job_location     460 non-null    object
 3   post_date        429 non-null    object
 4   salary           460 non-null    object
 5   job_url          460 non-null    object
 6   job_description  444 non-null    object
 7   Rate_by          460 non-null    object
dtypes: object(8)
memory usage: 32.3+ KB


0                            NaN
1                            NaN
2       $7,349 - $10,000 a month
3     $109,000 - $166,000 a year
4     $151,736 - $178,900 a year
                 ...            
95                           NaN
96                     Full-time
97      $66,000 - $87,500 a year
98                           NaN
99                           NaN
Name: salary, Length: 100, dtype: object

In [86]:
import string, re
salary_df[['low','high']] = salary_df.salary.str.split('-', expand=True)
salary_df[['Rate_by','low','high']]

,Rate_by,low,high
2,Monthly,"$7,349","$10,000 a month"
3,Yearly,"$109,000","$166,000 a year"
4,Yearly,"$151,736","$178,900 a year"
8,Yearly,"$96,900","$234,300 a year"
10,Yearly,"$99,900","$168,400 a year"
...,...,...,...
1082,Yearly,"$120,000","$180,000 a year"
1084,Yearly,"$80,000","$135,000 a year"
1088,Yearly,"$120,000","$185,000 a year"
1090,Hourly,$60,$65 an hour


In [88]:
def find_salary(a):
    if a:

        # Regular expression pattern to match digits and commas
        pattern = r'[\d,.]+'

        digits_and_commas = re.findall(pattern, a)

        numerical_value = ''.join(digits_and_commas)

        numerical_value = numerical_value.replace(',', '')

        return numerical_value
    else:
        return 0
    
salary_df['min'] = salary_df['low'].apply(find_salary)
salary_df['max'] = salary_df['high'].apply(find_salary)
salary_df.head(30)


,job_title,company,job_location,post_date,salary,job_url,job_description,Rate_by,low,high,min,max
2,WIC Data Scientist,Health & Human Services Comm,"Austin, TX 78728",Posted 15 days ago,"$7,349 - $10,000 a month",https://www.indeed.com/rc/clk?jk=a6a5fef4f8e96...,Data Scientist I \n \n Want to use your hig...,Monthly,"$7,349","$10,000 a month",7349,10000
3,"Data Scientist, Product Analytics",Facebook App,"Austin, TX",Posted 21 days ago,"$109,000 - $166,000 a year",https://www.indeed.com/rc/clk?jk=41e4c28301bed...,"As a Data Scientist at Meta, you will shape th...",Yearly,"$109,000","$166,000 a year",109000,166000
4,Sr. Data Scientist (Machine Learning Engineering),Petco,"Remote in Austin, TX 78701",Posted 6 days ago,"$151,736 - $178,900 a year",https://www.indeed.com/rc/clk?jk=aa51ef4db60e8...,C ome work at a place where innovation and tea...,Yearly,"$151,736","$178,900 a year",151736,178900
8,"Senior Data Scientist, Experimentation",PayPal,"Austin, TX",Posted 30+ days ago,"$96,900 - $234,300 a year",https://www.indeed.com/rc/clk?jk=7d90eaf637cd0...,"At PayPal (NASDAQ: PYPL), we believe that ever...",Yearly,"$96,900","$234,300 a year",96900,234300
10,Decision Scientist,Electronic Arts,"Hybrid remote in Austin, TX 78729",Posted 11 days ago,"$99,900 - $168,400 a year",https://www.indeed.com/rc/clk?jk=8d484aa5532be...,Decision Scientist\nEA’s Worldwide Customer Ex...,Yearly,"$99,900","$168,400 a year",99900,168400
12,Lead Data Scientist,Electronic Arts,"Austin, TX 78729",Posted 1 day ago,"$136,000 - $218,900 a year",https://www.indeed.com/rc/clk?jk=6b1d77d9d0c7b...,EA's Data & Analytics (DnA) team provides game...,Yearly,"$136,000","$218,900 a year",136000,218900
15,Distinguished Data Scientist,Torc Robotics,"Remote in Austin, TX",Posted 23 days ago,"$249,600 - $299,500 a year",https://www.indeed.com/rc/clk?jk=63122bff3bea9...,"About the Company\n At Torc, we have always be...",Yearly,"$249,600","$299,500 a year",249600,299500
16,Data Scientist,Plaxonic,"Austin, TX",EmployerActive 9 days ago,$45 - $50 an hour,https://www.indeed.com/company/PLAXONIC/jobs/D...,Data Science - Software Engineer\nLocation: Au...,Hourly,$45,$50 an hour,45,50
20,Machine Learning Engineer L-1,Lauretta AI LLC,"Austin, TX 78705 (Hancock area)",Posted 30+ days ago,"$80,000 - $93,000 a year",https://www.indeed.com/rc/clk?jk=73a077ff4554a...,"Job Description:\n\nDevelop high-quality, main...",Yearly,"$80,000","$93,000 a year",80000,93000
21,"Senior Data Scientist, Growth Analytics",Realtor.com Careers,"Austin, TX",Posted 30+ days ago,"$135,100 - $196,100 a year",https://www.indeed.com/rc/clk?jk=f433ae766610b...,"At Realtor.com®, we have among the most compre...",Yearly,"$135,100","$196,100 a year",135100,196100
